In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly as py
import plotly.express as px
import numpy as np
import gmaps
import gmaps.datasets 
from folium import Map
from folium.plugins import HeatMap
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings('ignore')

# Because of the size of the dataset we must store the dataset in googlebigquery and then take the data as needed for our analysis

In [ ]:
#Set environment variables for your notebook
import pyarrow 
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/amandamanfredo/Downloads/liquor_data.json'
#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
bigquery_client = bigquery.Client()
#Write Query on BQ
QUERY = """
SELECT date,store_number,store_name,address,city,zip_code,store_location,
county,category_name,vendor_number,vendor_name,item_description,pack,
bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,
sale_dollars,volume_sold_liters
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE  (date>='2012-01-01' AND date<'2022-03-01') AND store_name LIKE '%Hy-Vee%'
ORDER BY rand()
LIMIT 500000
  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum().plot.bar(); 

In [ ]:
#the dataframe is large enough that we can ignore null values and still have plenty of useful data to analyze
df.dropna(inplace=True)
df.drop_duplicates()

In [ ]:
#seeing how much the dataframe changes after dropping null values
df.shape

In [ ]:
df.info(memory_usage="deep")

In [ ]:
#trying to decrease the storage usage of the datset

#decreasing integer storage
df['bottle_volume_ml'] = df['bottle_volume_ml'].astype('int32')
df['pack'] = df['pack'].astype('int32')
df['bottles_sold'] = df['bottles_sold'].astype('int32')

#decreasing float memory usage
df['state_bottle_cost'] = df['state_bottle_cost'].astype('float32')
df['state_bottle_retail'] = df['state_bottle_retail'].astype('float32')
df['sale_dollars'] = df['sale_dollars'].astype('float32')
df['volume_sold_liters'] = df['volume_sold_liters'].astype('float32')

In [ ]:
df.info(memory_usage="deep")

In [ ]:
df['Profit_per_bottle']=df['state_bottle_retail']-df['state_bottle_cost']
df['Profit_per_sale']=df['Profit_per_bottle']*df['bottles_sold']
df.head(20)

In [ ]:
#getting an idea of the average profit per bottle
print('mean:',df['Profit_per_bottle'].mean())

#getting an idea of median profit since a few expensive/profitable bottles can skew mean
print('median:',df['Profit_per_bottle'].median())

#finding the max profit per bottle
df.loc[df['Profit_per_bottle'] == df['Profit_per_bottle'].max()] 

In [ ]:
#seeing the distribution of profit per bottle
df.hist(column='Profit_per_bottle', bins=100, grid=False, figsize=(12,8),range=[0, 50])

In [ ]:
df['category_name'].nunique()

In [ ]:
#turning the store names to lowercase to prevent discrepancies
df['store_name'] = df['store_name'].str.lower()

df['category_name'] = df['category_name'].str.lower()

#turning the county names to lowercase to prevent discrepancies
df['county'] = df['county'].str.lower()

df['city'] = df['city'].str.lower()

df['vendor_name'] = df['vendor_name'].str.lower()

In [ ]:
#defining a function to create a column of the different store types for hy-vee
def store(b):
    if ('quick' in b) or ('kwik' in b) or ('conven' in b) or ('fresh' in b) or ('c-store' in b):
        return 'Convenience'
    elif ('gas' in b) or ('petro' in b) or ('fuel' in b):
        return 'Gas'
    elif ('liquor' in b) or ('spirits' in b) or ('beverage' in b) or ('bottle' in b) or ('distil' in b) or ('wine' in b):
        return 'Liquor Store'
    elif ('drug' in b) or ('pharmacy' in b):
        return 'Drug Store'
    else:
        return 'Supermarket'

df['Store Type'] = df.store_name.apply(lambda y: store(y))

In [ ]:
df['Store Type'].value_counts()

In [ ]:
#making a function to classify the different liquor types into a column

def func(a):
    if "vodka" in a:
        return "vodka"
    elif "tequila" in a:
        return "tequila"
    elif "rum" in a:
        return "rum"
    elif "gin" in a:
        return "gin"
    elif "wine" in a:
        return "wine"
    elif "beer" in a:
        return "beer"
    elif "wine" in a:
        return "wine"
    elif "brand" in a:
        return "brandy"
    elif ('liqueur' in a) or ("triple" in a):
        return 'liqueurs'
    elif "schnap" in a:
        return "schnapps"
    elif ('grain spirits' in a):
        return 'moonshine'
    elif ('mezcal' in a):
        return 'mezcal'
    elif ('cocktail' in a):
        return 'cocktail mix'
    elif 'special' in a:
        return "gifts"
    elif "whisk" in a or "bourbon" in a or "scotch" in a:
        return "whisky"
    else:
        return "Other"

df["LiquorType"] = df.category_name.apply(lambda x: func(x))

In [ ]:
df['LiquorType'].value_counts()

In [ ]:
#turing the column to str so we can make this function
df.bottle_volume_ml = df.bottle_volume_ml.astype(str)

In [ ]:
#making a function to convert bottle volume into categories
def bottle(c):
    if "750" in c:
        return "750"
    elif "1750" in c:
        return "1750"
    elif "375" in c:
        return "375"
    elif "50" in c:
        return "c"
    elif "1000" in c:
        return "1000"
    elif "200" in c:
        return "200"
    elif "100" in c:
        return "100"
    else:
        return "Other"

df['Bottle Size'] = df.bottle_volume_ml.apply(lambda y: bottle(y))

In [ ]:
#dropping original column since its now repetitive
df = df.drop('bottle_volume_ml', 1)

In [ ]:
df.head()

# Looking at the type of store generating the most profits

In [ ]:
total_sale_store = df.groupby('Store Type').agg({'Profit_per_sale':'sum'}).reset_index().sort_values('Profit_per_sale',ascending =False)
tts = total_sale_store.head(50)
tts

In [ ]:
#profits generated by store type
a1 = (10, 10)
fig, ax = plt.subplots(figsize=a1)
plot= sns.barplot(x="Store Type", y="Profit_per_sale", data=tts,palette ="deep")
plot.set_xticklabels(ax.get_xticklabels(),rotation=90)
plot.set_title('Best Store Type by Profits')
ax.set_ylabel('Profits (Dollars)')
ax.set_xlabel('Store Type')

# Which liquor type are most profitable overall 

In [ ]:
total_sale_liquor = df.groupby('LiquorType').agg({'Profit_per_sale':'sum'}).reset_index().sort_values('Profit_per_sale',ascending =False)
ttb = total_sale_liquor.head(10)
ttb

In [ ]:
#total profits generated by each liquor type
fig, ax = plt.subplots(figsize=(10,10))
plot= sns.barplot(x="LiquorType", y="Profit_per_sale", data=ttb,palette ="deep")
plot.set_xticklabels(ax.get_xticklabels(),rotation=50)
plot.set_title('Most Profitable Liquor Types ')
ax.set_ylabel('Profit (Dollars)')
ax.set_xlabel('Liquor Type')

In [ ]:
# Draw Plots comparing the how much the state buys each liquor for, and how much they sell it for
# fig, axes = plt.subplots(1, 2, figsize=(20,7),sharey=True, sharex=True) #, dpi= 80, ylim=(0,80))
plt.ylim(0,80)
plt.xticks(rotation=45)
sns.boxplot(x='LiquorType', y='state_bottle_cost', data=df)

In [ ]:
sns.boxplot(x='LiquorType', y='state_bottle_retail', data=df)
plt.ylim(0,80)
plt.xticks(rotation=45)

In [ ]:
sns.boxplot(x='LiquorType', y='Profit_per_bottle', data=df)
plt.ylim(0,50)
plt.xticks(rotation=45)

In [ ]:
#boxplot of avg profit per sale based on liquor type
sns.boxplot(x='LiquorType', y='Profit_per_sale', data=df)
plt.ylim(0,300)
plt.xticks(rotation=45)

In [ ]:
# Draw Plot of profits per sale by store type

plt.xticks(rotation=45)
sns.boxplot(x='Store Type', y='Profit_per_sale', data=df, linewidth=1.5)
plt.ylim(0,200)


In [ ]:
supermarket=df[df['Store Type']=='Supermarket']
supermarket.head()

In [ ]:
#plotting liquor sales for supermarkets
plt.xticks(rotation=45)
sns.barplot(x='LiquorType', y='sale_dollars', data=supermarket, linewidth=1.5)
plt.ylim(0,250)
plt.title("Supermarket Sales ($)")
sns.set_style("white")

In [ ]:
liquorst=df[df['Store Type']=='Liquor Store']

In [ ]:
#plotting liquor store sales by liquor type
plt.xticks(rotation=45)
sns.barplot(x='LiquorType', y='sale_dollars', data=liquorst)
plt.ylim(0,300)
plt.title('Liquor Store Sales ($)')


In [ ]:
gas=df[df['Store Type']=='Gas']

In [ ]:
#plotting gas station sales by liquor type
plt.xticks(rotation=45)
sns.barplot(x='LiquorType', y='sale_dollars', data=gas)
plt.ylim(0,150)
plt.title('Gas Station Sales ($)')

In [ ]:
#comparing store type for all categories
desm=df[df['city']=='des moines']
desm=desm.groupby('Store Type').sum()
desm

In [ ]:
plt.xticks(rotation=45)
sns.barplot(x='Store Type', y='sale_dollars', data=des)
plt.ylim(0,150)
plt.title('Sales ($) By Store Type in Des Moines')

# Exploring which categories of alcohol are more profitable on average

In [ ]:
best10 = df.groupby(['category_name'])['Profit_per_bottle'].mean().groupby(['category_name']).max().sort_values().groupby(['category_name']).sum().sort_values(ascending=False).reset_index()

best10_plot = px.bar(best10.head(10),x=best10['category_name'].head(10), y='Profit_per_bottle',color='Profit_per_bottle')
best10_plot.update_layout(
    title="10 Best Average Profit per Bottle Liquor Categories",
    xaxis_title="Category Name",
    yaxis_title="Average Profit per Bottle($)")
best10_plot.show()

# Exploring which categories of alcohol are least profitable

In [ ]:
low10_plot = px.bar(best10.tail(10),x=best10['category_name'].tail(10), y='Profit_per_bottle',color='Profit_per_bottle')
low10_plot.update_layout(
    title="10 Lowest Profit Liquor Categories",
    xaxis_title="Category Name",
    yaxis_title="Average Profit per Bottle($)")
low10_plot.show()

# Exploring which counties have the highest profits overall 

In [ ]:
bestcounty = df.groupby(['county'])['Profit_per_sale'].sum().groupby(['county']).max().sort_values().groupby(['county']).sum().sort_values(ascending=False).reset_index()

bestcounty_plot = px.bar(bestcounty.head(10),x=bestcounty['county'].head(10), y='Profit_per_sale',color='Profit_per_sale')
bestcounty_plot.update_layout(
    title="10 Best County Profit Liquor",
    xaxis_title="county",
    yaxis_title="Total Profits ($)")
bestcounty_plot.show()

# Comparing sales profits by city

In [ ]:
bestcity = df.groupby(['city'])['Profit_per_sale'].sum().groupby(['city']).max().sort_values().groupby(['city']).sum().sort_values(ascending=False).reset_index()

bestcity_plot = px.bar(bestcity.head(10),x=bestcity['city'].head(10), y='Profit_per_sale',color='Profit_per_sale')
bestcity_plot.update_layout(
    title="10 Best Cities for Profit Liquor Categories",
    xaxis_title="City",
    yaxis_title="Total Profits ($)")
bestcity_plot.show()

# Finding Total Bottles Sold in each City

In [ ]:
bottlecity = df.groupby(['city'])['bottles_sold'].sum().groupby(['city']).max().sort_values().groupby(['city']).sum().sort_values(ascending=False).reset_index()

bottlecity_plot = px.bar(bottlecity.head(10),x=bottlecity['city'].head(10), y='bottles_sold',color='bottles_sold')
bottlecity_plot.update_layout(
    title="10 Highest Bottles Sold by City",
    xaxis_title="City",
    yaxis_title="Total Bottles Sold")
bottlecity_plot.show()

# Comparing Vendors with the Highest Profits

In [ ]:
bestvendors = df.groupby(['vendor_name'])['Profit_per_sale'].sum().groupby(['vendor_name']).max().sort_values().groupby(['vendor_name']).sum().sort_values(ascending=False).reset_index()

bestvendors_plot = px.bar(bestvendors.head(10),x=bestvendors['vendor_name'].head(10), y='Profit_per_sale',color='Profit_per_sale')
bestvendors_plot.update_layout(
    title="10 Best Vendors for Profit Liquor Categories",
    xaxis_title="Vendor",
    yaxis_title="Total Profits ($)")
bestvendors_plot.show()

In [ ]:
#finding number of purchases by storetype in each city
df.groupby(['city'])['Store Type'].value_counts()

In [ ]:
#finding the most commonly purchased category
df['category_name'].mode()

In [ ]:
#finding the most common vendor in the dataset
df['vendor_name'].mode()

# Looking at the sales for each store location

In [ ]:
store_sales= df.groupby('store_name').sale_dollars.sum().groupby(['store_name']).max().sort_values().groupby(['store_name']).sum().sort_values(ascending=False).reset_index()
store_sales_plot = px.bar(store_sales.head(10),x=store_sales['store_name'].head(10), y='sale_dollars',color='sale_dollars')
store_sales_plot.update_layout(
    title="10 Highest Sales Stores",
    xaxis_title="Store Number",
    yaxis_title="Total Sales($)")
store_sales_plot.show()

# Checking if the stores with the highest sales also have the highest profits

In [ ]:
store_profit= df.groupby('store_name').Profit_per_sale.sum().groupby(['store_name']).max().sort_values().groupby(['store_name']).sum().sort_values(ascending=False).reset_index()
store_profit_plot = px.bar(store_profit.head(10),x=store_profit['store_name'].head(10), y='Profit_per_sale',color='Profit_per_sale')
store_profit_plot.update_layout(
    title="10 Highest Profit Stores",
    xaxis_title="Store Name",
    yaxis_title="Total Profits($)")
store_profit_plot.show()

# Looking at monthly profits for liquor types

# Creating a Heatmap of the locations of the liquor stores in Iowa

In [ ]:
#Changing the column type to string so we can strip leading POINT
df['store_location'] = df.store_location.astype(str)

#stripping the non-latitude-longitude values from the column
df['store_location'] = df['store_location'].map(lambda x: x.lstrip('POINT (').rstrip(')'))

#creating latitude and longitude columns to use for our heatmap
df[['Longitude','Latitude']] = df['store_location'].str.split(' ', expand=True)


In [ ]:
#folium requires the data to be float type

df['Latitude'] = df['Latitude'].str.replace(' ', '').astype(float)
df['Longitude'] = df['Longitude'].str.replace(' ', '').astype(float)

#dropping the original store location column since now it is repetitive
df = df.drop('store_location', 1)

In [ ]:
df.head()

In [ ]:
import folium
from IPython.display import display

mymap= folium.Map(location=[43.088341,-93.990256], zoom_start=8, )

hm_wide = HeatMap(
    list(zip(df.Latitude.values, df.Longitude.values)),
    min_opacity=0.3,
    radius=10, 
    blur=9, 
    max_zoom=1,
)
mymap.add_child(hm_wide)
#mymap.add_child(folium.ClickForMarker(popup='Potential Location'))

In [ ]:
# university of iowa=41.6627, 91.5550
# iowa state= 42.0267, 93.6465
# univerisity northern iowa=42.5122, 92.4646
# kirkwood comminuty collegee=41.9110, 91.6522
# drake university= 41.6031, 93.6546

# # dictionary with list object in values
# details = {
#     'School' : ['university of iowa', 'iowa state', 'univerisity northern iowa', 'kirkwood community college','drake university'],
#     'lat' : [41.6627, 42.0267, 42.5122, 41.9110,41.6031],
#     'lon' : [91.5550, 93.6465,92.4646, 91.6522,93.6546],
# }
  
# # creating a Dataframe object 
# college = pd.DataFrame(details)
# college

# # add marker one by one on the map
# for i in range(0,len(college)):
#    folium.Marker(
#       location=[college.iloc[i]['lat'], college.iloc[i]['lon']],
#       popup=college.iloc[i]['School'],
#    ).add_to(mymap)

# # Show the map again
# mymap

 

# Looking at time related data 

In [ ]:
#making a new dataframe I can make changes to
timedf=df
#converting date column to datetime
timedf['date'] = pd.to_datetime(timedf['date'])
timedf['day_name'] = timedf['date'].dt.day_name()
timedf['year']=timedf['date'].dt.year

In [ ]:
from collections import defaultdict, OrderedDict

data = defaultdict(list)
for r in timedf.itertuples():
    data[r.year].append([r.Latitude, r.Longitude])
    
data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))

In [ ]:
import folium
from IPython.display import display
from folium.plugins import HeatMapWithTime

m = folium.Map([43.088341,-93.990256],
               tiles='stamentoner',
               zoom_start=7)


hm = HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()), 
                     radius=10,
                     auto_play=True,
                     min_opacity=0.3)
hm.add_to(m)
#m.save(outfile= "hyve.html")
m

In [ ]:
import folium
from IPython.display import display
from folium.plugins import HeatMapWithTime

m = folium.Map([43.088341,-93.990256],
               tiles='openstreetmap',
               zoom_start=7)


hm = HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()), 
                     radius=10,
                     auto_play=True,
                     min_opacity=0.3)
hm.add_to(m)
#m.save(outfile= "timemap.html")
m

In [ ]:
timedf.head()

In [ ]:
#finding the total sales profits each day
#dailyprofits = timedf.groupby([timedf.date.dt.dayofyear])['Profit_per_sale'].sum().reset_index()
dailyprofits = timedf.groupby(['date'])['Profit_per_sale'].sum().reset_index()

#rename sum of profit per sale to profit per day
dailyprofits.rename({'Profit_per_sale': 'Profit_per_day'}, axis=1, inplace=True)
dailyprofits.head()

In [ ]:
#finding which day had the highest profit sales
print("Max profits:", dailyprofits.loc[dailyprofits['Profit_per_day'] == dailyprofits['Profit_per_day'].max()])

#finding the day with the lowest profit sales
print("Min profits:",dailyprofits.loc[dailyprofits['Profit_per_day'] == dailyprofits['Profit_per_day'].min()])

In [ ]:
# Draw Plot
fig, axes = plt.subplots(1, 2, figsize=(20,7)) #, dpi= 80, ylim=(0,80))
plt.ylim(0,1000)
plt.xticks(rotation=45)
sns.barplot(x='day_name', y='sale_dollars', data=timedf, ax=axes[0])
sns.boxplot(x='year', y='volume_sold_liters', data=timedf)

In [ ]:
y=dailyprofits['Profit_per_day']
x=dailyprofits['date']
plt.figure(figsize=(15,8))
sns.lineplot(x,y)
plt.title('Profit per Day')

In [ ]:
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(dailyprofits, x=dailyprofits.date, y=dailyprofits.Profit_per_day, title='Daily Profits 2014-2022 ')    

# Looking at bottles sold per day

In [ ]:
#creating a dataframe of the total bottles sold per day
dailybottles=timedf.groupby(['date'])['bottles_sold'].sum().reset_index()

#dailysales = dailysales.set_index('date')

#rename sum of profit per sale to profit per day
dailybottles.rename({'bottles_sold': 'daily_bottles_sold'}, axis=1, inplace=True)

#converting the date to datetime format
#dailybottles['date']=pd.to_datetime(dailybottles['date'])
dailybottles.head()

In [ ]:
#plotting the bottles sold per day
y=dailybottles['daily_bottles_sold']
x=dailybottles['date']
plt.figure(figsize=(15,8))
sns.lineplot(x,y)
plt.title('Bottles Sold Per Day')

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(x,y)
plt.title('Daily Bottles Sold')

# Comparing weekly sales

In [ ]:
#making a new df to experiment with without ruining original
df2=df

#converting date column to datetime
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
#finding the amount of bottles of liquor sold each week
weekly = df2.groupby(df2.date.dt.weekofyear)['bottles_sold'].agg('sum').reset_index()

#rename sum of bottles sold to weekly bottles sold
weekly.rename({'bottles_sold': 'weekly_bottles_sold'}, axis=1, inplace=True)

#plotting 
x1=weekly['date']
y1=weekly['weekly_bottles_sold']
plt.figure(figsize=(10,8))
sns.barplot(x1,y1)
plt.title('Bottles Sold Each Week')

In [ ]:
#lineplot of the weekly bottle sales
plt.figure(figsize=(10,8))
sns.lineplot(x1,y1)

In [ ]:
#finding the profit each week
weekly = df2.groupby(df2.date.dt.weekofyear)['Profit_per_sale'].agg('sum').reset_index()


#rename sum of bottles sold to weekly bottles sold
weekly.rename({'Profit_per_sale': 'weekly_profit'}, axis=1, inplace=True)

#plotting the profits by week
xw=weekly['date']
yw=weekly['weekly_profit']
plt.figure(figsize=(10,8))
sns.barplot(xw,yw)
plt.title('Weekly Profit Each Week')

In [ ]:
plt.figure(figsize=(10,8))
sns.lineplot(xw,yw)

In [ ]:
y = df.groupby('date')['Profit_per_sale'].sum().resample('MS').mean()
y.plot()

# Looking at months overall for seasonal patterns

In [ ]:
#finding the amount of bottles of liquor sold each month
monthly = df2.groupby(df2.date.dt.month)['bottles_sold'].agg('sum').reset_index()

#rename sum of bottles sold to monthly bottles sold
monthly.rename({'bottles_sold': 'monthly_bottles_sold'}, axis=1, inplace=True)

#plotting 
x1=monthly['date']
y1=monthly['monthly_bottles_sold']
plt.figure(figsize=(10,8))
sns.lineplot(x1,y1)
plt.title('Bottles Sold Each Month')

# Looking at months by year for overall trends

In [ ]:
timedf['year_month'] = timedf['date'].dt.strftime('%Y-%m')

In [ ]:
vodka = timedf[timedf['LiquorType'] == 'vodka'].groupby('year_month', as_index=False)['Profit_per_sale'].sum()
#print(vodka)

vodx=vodka['year_month']
vody=vodka['Profit_per_sale']

sns.lineplot(vodx,vody,data=vodka)
#plt.axhline(vodka['Profit_per_sale'].mean(),color="k",linestyle="dotted",linewidth=2)

plt.title('Vodka monthly sales') 

whisk = timedf[timedf['LiquorType'] == 'whisky'].groupby('year_month', as_index=False)['Profit_per_sale'].sum()
whisky=whisk['Profit_per_sale']

rum = timedf[timedf['LiquorType'] == 'rum'].groupby('year_month', as_index=False)['Profit_per_sale'].sum()
rumy=rum['Profit_per_sale']

liqueurs = timedf[timedf['LiquorType'] == 'liqueurs'].groupby('year_month', as_index=False)['Profit_per_sale'].sum()
rumy=rum['Profit_per_sale']

top4=[vody, whisky, rumy, liqueurs]
headers = ["Vodka","Whiskies","Rum", "Liqueurs"]

top4_sell = pd.concat(top4, axis=1, keys=headers)
top4_sell.index = top4_sell.index +1

import matplotlib.ticker as ticker

ax = top4_sell.plot(linewidth=1,fontsize=8);
ax.set_xlabel('Month and Year');
ax.legend(fontsize=8)
ax.xaxis.set_major_locator(ticker.MultipleLocator(20));

In [ ]:
#finding the amount of profit of liquor sold each month
monthyr = timedf.groupby(timedf['year_month'])['Profit_per_sale'].agg('sum').reset_index()

#rename sum of profit sold to monthly bottles sold
monthyr.rename({'Profit_per_sale': 'monthyr_profit'}, axis=1, inplace=True)

#plotting 
xm=monthyr['year_month']
ym=monthyr['monthyr_profit']
plt.figure(figsize=(15,8))
sns.lineplot(xm,ym)
plt.title('Profit Each Month')
plt.xticks(rotation=90);
#plt.autofmt_xdate()

In [ ]:
#monthyr.set_index('year_month', inplace=True)
monthyr['year_month']=pd.to_datetime(monthyr['year_month'])

In [ ]:
monthyr.set_index('year_month', inplace=True)

In [ ]:
monthyr.plot()

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

In [ ]:
#can make a function to find adf value
def adftest(series):
    results=adfuller(series)
    print(f'ADF Statistic: {results[0]}')
    print(f'p-value: {results[1]}')
    if results[1] <= 0.05:
        print('strong evidence againstnull hypothesis. Data is stationary')
    else:
        print('weak evidence against null hypothesis. Data is NON stationary')
        
adftest(monthyr['monthyr_profit'])

In [ ]:
#Detrending a time series is to remove the trend component from a time series

# Using scipy: Subtract the line of best fit
from scipy import signal
detrended = signal.detrend(monthyr.monthyr_profit.values)
plt.plot(detrended)
plt.title('Liquor Sales detrended by subtracting the least squares fit', fontsize=16)


In [ ]:
# when there is a strong seasonal pattern, the ACF plot usually reveals 
# definitive repeated spikes at the multiples of the seasonal window. 

from pandas.plotting import autocorrelation_plot
plt.rcParams.update({'figure.figsize':(9,5), 'figure.dpi':120})
autocorrelation_plot(monthyr.monthyr_profit.tolist())

# #Differencing the Timeseries data

In [ ]:
#Viewing the data in each state of differencing with its autocorrelation plot
# If a series is significantly autocorrelated, that means, the previous values of the series (lags) 
# may be helpful in predicting the current value. Partial Autocorrelation also conveys similar 
# information but it conveys the pure correlation of a series and its lag, excluding the correlation 
# contributions from the intermediate lags.


from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# Original Series
fig, axes = plt.subplots(3, 2, figsize=(20,10))
axes[0, 0].plot(monthyr.monthyr_profit); axes[0, 0].set_title('Original Series')
plot_acf(monthyr.monthyr_profit, ax=axes[0, 1], lags=52)

# 1st Differencing
axes[1, 0].plot(monthyr.monthyr_profit.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(monthyr.monthyr_profit.diff().dropna(), ax=axes[1, 1], lags=52)

# 2nd Differencing
axes[2, 0].plot(monthyr.monthyr_profit.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(monthyr.monthyr_profit.diff().diff().dropna(), ax=axes[2, 1], lags=52)


plt.show()

In [ ]:
#Testing if data is stationary after first difference= it is
adftest(monthyr.monthyr_profit.diff().dropna())

In [ ]:
# PACF plot of 1st differenced series
fig, axes = plt.subplots(1, 2, figsize=(20,5))
axes[0].plot(monthyr.monthyr_profit.diff()); axes[0].set_title('1st Differencing')
plot_pacf(monthyr.monthyr_profit.diff().dropna(), ax=axes[1])

plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,5))
axes[0].plot(monthyr.monthyr_profit.diff()); axes[0].set_title('1st Differencing')
plot_acf(monthyr.monthyr_profit.diff().dropna(), ax=axes[1])
plt.show()


In [ ]:
#p= 3 based on PACF, d=1 differencing once, q=1 from autocorrelation plot
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(monthyr.monthyr_profit,
              order=(2,1,2))
model_fit = model.fit()  #model_fit = model.fit(disp=0)
print(model_fit.summary())


In [ ]:
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2, figsize=(15,5))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Actual vs Fitted
monthyr['forecast']=model_fit.predict(dynamic=False)
monthyr[['monthyr_profit','forecast']].plot(figsize=(12,8), linewidth=1.5)
plt.ylabel('Sale Profits')

In [ ]:
#another way to plot actual vs predicted that hopefully shows forecast
from pandas.tseries.offsets import DateOffset
pred_date=[monthyr.index[-1]+ DateOffset(months=x)for x in range(0,24)]
# Giving similar names to columns.

# Input:

pred_date=pd.DataFrame(index=pred_date[1:],columns=monthyr.columns)
pred_date

# monthyr['diff']=monthyr.monthyr_profit.diff().dropna()

monthyr=pd.concat([monthyr,pred_date])
monthyr['forecast'] = model_fit.predict(start = 90, end = 120, dynamic= False)
monthyr[['monthyr_profit', 'forecast']].plot(figsize=(12, 8), linewidth=1.5)

In [ ]:
#pred = model_fit.get_prediction(start=pd.to_datetime('2020-01-01'), dynamic=False)

#update your data with the real values instead, you set dynamic=False
pred = model_fit.get_prediction(start=pd.to_datetime('2020-01-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = monthyr.monthyr_profit['2012':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Profits')
plt.legend()
plt.show()


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(monthyr.monthyr_profit,order=(2,1,2), exogeneous=monthyr.index,
              seasonal_order=(2,1,2,12),enforce_stationarity=True,
              enforce_invertibility=False)
results = model.fit()

In [ ]:
#plotting the seasonal forecasted sales
forecast = results.predict(dynamic=False)
forecast[1:].plot(linewidth=1.5,legend=True)
monthyr.monthyr_profit.plot(linewidth=1.5,legend=True)


In [ ]:
#plotting the predictions with confidence interval
pred = results.get_prediction(start=pd.to_datetime('2020-01-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = monthyr.monthyr_profit['2012':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Profits')
plt.legend()
plt.show()

# Stepwise Regression for important features

In [ ]:
dfr=df.copy()

In [ ]:
#separate types of columns/features

categorical_features=['LiquorType', 'Store Type', 'Bottle Size', 'county', 'city']
numerical_features = ['pack', 'state_bottle_cost', 'state_bottle_retail','bottles_sold','volume_sold_liters']
                   #  'Longitude', 'Latitude']

numerical=dfr[numerical_features]
categorical=dfr[categorical_features]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# transform data
scaled = scaler.fit_transform(dfr[numerical_features])
dfr[numerical_features]=scaled

In [ ]:
#dummifying the categorical features
dummy= pd.get_dummies(dfr[categorical_features], drop_first=True)

#viewing dummy to make sure the values are correct
dummy

In [ ]:
x=pd.concat([dummy, dfr[numerical_features]], axis=1)
yols = np.log(df['Profit_per_sale']+1)

In [ ]:
# import numpy as np
# import statsmodels.api as sm
# from sklearn.model_selection import train_test_split

# xols=sm.add_constant(x)

# X_train, X_test, Y_train, Y_test=train_test_split(xols,yols, test_size=.3)
#
# #   
# results=sm.OLS(Y_train,X_train).fit()
# print(results.summary())
# 

In [ ]:
# from sklearn import linear_model
# X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=.3)
# lm = linear_model.LinearRegression() 
# #generate fit of model base on training set

# model = lm.fit(X_train, Y_train) 
# #generate predicted values of y_test from X_test based off of training set
# predictions = lm.predict(X_test)
# #plotting predicted ys against y values in test set 
# plt.scatter(Y_test, predictions)
# lm.score(X_test, Y_test)

# LassoCV Regession

Seeing if Lasso can help with feature selection

In [ ]:
x.isnull().sum()

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
lasso = LassoCV(alphas=np.arange(0, 1, 0.01), cv=cv, n_jobs=-1)

#fitting the model
lasso.fit(X_train, Y_train)

lasso.score(X_train, Y_train)

In [ ]:
lasso.score(X_test, Y_test)
#big difference between train and test set indicating overfitting

In [ ]:
# Plot important coefficients of 2nd model
coefs1 = pd.Series(lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coefs1 != 0)) + " features and eliminated the other " +  \
      str(sum(coefs1 == 0)) + " features")
imp_coefs = pd.concat([coefs1.sort_values().head(10),
                     coefs1.sort_values().tail(10)])
imp_coefs.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model", fontsize=25)
plt.show()

#all the top coefficients are location related- county and city

In [ ]:
# from sklearn import svm

# classifiers = [
#     svm.SVR(),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
#     linear_model.LinearRegression()]


# for item in classifiers:
#     print(item)
#     clf = item
#     clf.fit(X_train, Y_train)
#     print(clf.score(X_train, Y_train),'\n')
#     print(clf.score(X_test, Y_test),'\n')

# K Means Clustering

 “the objective of K-means is simple: group similar data points together and discover underlying patterns"

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.datasets import make_blobs

kmeans_kwargs = {
 "init": "random", "n_init": 10,"max_iter": 300,"random_state": 42}


sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(x)
    sse.append(kmeans.inertia_)

#plotting to see optimal number of clusters
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

#using function to see optimal number of clusters
kl = KneeLocator(
range(1, 11), sse, curve="convex", direction="decreasing")
k=kl.elbow
k


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#carrying out kmeans with the correct number of clusters
kmeans = KMeans(n_jobs = -1, n_clusters = 3, init='k-means++')

#fitting the model
kmeans.fit(x)

#creating dataframe to add the predictions to
pred = kmeans.predict(x)
frame = pd.DataFrame(x)
frame['cluster'] = pred

#getting the amount of samples in each cluster
frame['cluster'].value_counts()

In [ ]:
labels = kmeans.labels_
centers = kmeans.cluster_centers_
plt.scatter(x.iloc[:, 19], x.iloc[:, 20], c = labels, alpha = 0.8)
plt.scatter(centers[:, 5], centers[:, 5], marker = '+', s = 1000, c = [0, 1, 2,3,4,])
plt.show()

# Tree Based models

prepping the training data /n
features for tree models do not need to be standardized 

In [ ]:
#Recreating the numerical and categorical features we have selected
num=df[numerical_features]
categ=df[categorical_features]


In [ ]:
#encoding the categories for processing
for col_name in categ.columns:
    if(categ[col_name].dtype == 'object'):
        categ[col_name]= categ[col_name].astype('category')
        categ[col_name] = categ[col_name].cat.codes
categ

In [ ]:
#creating the x and y data to train and test
treex=pd.concat([num, categ], axis=1)
treey=df['Profit_per_sale']

# Decision Tree Regression

Builds regression or classification models in the form of a tree structure
Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees.


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

#splitting the data
Xt_train, Xt_test, yt_train, yt_test = train_test_split(treex,treey, 
test_size=0.30, random_state=0)

dt = DecisionTreeRegressor(random_state=0, criterion="mse")
dt.fit(Xt_train, yt_train)

dt_scores = cross_val_score(dt, Xt_train, yt_train, cv = 10)
print("mean cross validation score: {}".format(np.mean(dt_scores)))
print("score without cv: {}".format(dt.score(Xt_train, yt_train)))

# R^2 score on the test set
from sklearn.metrics import r2_score
print(r2_score(yt_test, dt.predict(Xt_test)))
print(dt.score(Xt_test, yt_test))


In [ ]:
#looking at the individual cv scores
dt_scores

In [ ]:
#plotting the decision tree model

dt_fit=dt.fit(Xt_train, yt_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(10,8), dpi=150)
plot_tree(dt_fit, feature_names=treex.columns, filled=True);

In [ ]:
dt = DecisionTreeRegressor(random_state=0, criterion="mse",max_depth=8, min_samples_split=8 )
dt.fit(Xt_train, yt_train)

dt_scores = cross_val_score(dt, Xt_train, yt_train, cv = 10)
print("mean cross validation score: {}".format(np.mean(dt_scores)))
print("score without cv: {}".format(dt.score(Xt_train, yt_train)))

# R^2 score on the test set
from sklearn.metrics import r2_score
print(r2_score(yt_test, dt.predict(Xt_test)))
print(dt.score(Xt_test, yt_test))

dt_fit=dt.fit(Xt_train, yt_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(10,8), dpi=150)
plot_tree(dt_fit, feature_names=treex.columns, filled=True);

In [ ]:
# from sklearn.model_selection import GridSearchCV
# params = {'max_depth': [2,4,6,8,10,12],
#          'min_samples_split': [2,3,4],
#          'min_samples_leaf': [2,4]}

# clf = DecisionTreeRegressor()
# gcv = GridSearchCV(estimator=clf,param_grid=params)
# gcv.fit(Xt_train,yt_train)

# model = gcv.best_estimator_
# model.fit(Xt_train,yt_train)
# dt2_fit=model.fit(Xt_train,yt_train)
# y_train_pred = model.predict(Xt_train)
# y_test_pred = model.predict(Xt_test)

# print(f'Train score {accuracy_score(y_train_pred,yt_train)}')
# print(f'Test score {accuracy_score(y_test_pred,yt_test)}')
# plt.figure(figsize=(10,8), dpi=150)
# plot_tree(dt2_fit, feature_names=treex.columns, filled=True);

# Random Forest Regression
-Combines the predictions from multiple machine learning algorithms together to make more accurate predictions than any individual model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#creating the tree
rf = RandomForestRegressor(random_state=0, criterion='mse')
rf.fit(Xt_train, yt_train)

#performing cross validation
rf_scores = cross_val_score(rf, Xt_train, yt_train, cv = 5)
print("mean cross validation score: {}".format(np.mean(rf_scores)))
print("score without cv: {}".format(rf.score(Xt_train, yt_train)))

rffit=rf.fit(Xt_train, yt_train)
# Scores on the test or hold-out set
from sklearn.metrics import r2_score
print(r2_score(yt_test, rf.predict(Xt_test)))
print(rf.score(Xt_test, yt_test))

# mean cross validation score: 0.9345547572053438
# score without cv: 0.9901271896815937
# 0.9026091822325131
# 0.9026091822325131

In [ ]:
# '''Running a comprehensive Gridsearch for Random Forest'''
# from sklearn.model_selection import GridSearchCV
# #Creating the parameters for gridsearch
# randomparams={'bootstrap': [True, False],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt'],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}



# rfcv = GridSearchCV(estimator = rf, param_grid = randomparams, cv = 3, n_jobs = -1)

# rfcv.fit(Xt_train, yt_train)
# print(r2_score(y_test, rfcv.predict(Xt_test)))
# print(rfcv.score(Xt_test, yt_test))